In [1]:
from pyspark.sql.types import StructType, StructField, StringType, LongType, DecimalType, DoubleType
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession \
    .builder \
    .config("fs.s3a.access.key", "Q3AM3UQ867SPQQA43P2F") \
    .config("fs.s3a.secret.key", "zuf+tfteSlswRu7BJ86wekitnifILbZam1KYY3TG") \
    .config("fs.s3a.endpoint", "https://play.min.io:9000") \
    .appName("VolumeCalculation") \
    .getOrCreate()

sc = spark.sparkContext

sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.attempts.maximum", "1")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.establish.timeout", "5000")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.timeout", "10000")

In [4]:
token_transfers_schema = StructType([ \
    StructField("token_address", StringType(), True), \
    StructField("from_address", StringType(), True), \
    StructField("to_address", StringType(), True), \
    StructField("value", DecimalType(38, 0), True), \
    StructField("transaction_hash", StringType(), True), \
    StructField("log_index", LongType(), True), \
    StructField("block_number", LongType(), True) \
  ])

In [5]:
transactions_schema = StructType([ \
    StructField("hash", StringType(), True), \
    StructField("nonce", LongType(), True), \
    StructField("block_hash", StringType(), True), \
    StructField("block_number", LongType(), True), \
    StructField("transaction_index", LongType(), True), \
    StructField("from_address", StringType(), True), \
    StructField("to_address", StringType(), True), \
    StructField("value", DecimalType(38, 0), True), \
    StructField("gas", LongType(), True), \
    StructField("gas_price", LongType(), True), \
    StructField("input", StringType(), True), \
    StructField("block_timestamp", LongType(), True), \
    StructField("max_fee_per_gas", LongType(), True), \
    StructField("max_priority_fee_per_gas", LongType(), True), \
    StructField("transaction_type", LongType(), True) \
  ])

In [6]:
receipts_schema = StructType([ \
    StructField("transaction_hash", StringType(), True), \
    StructField("transaction_index", LongType(), True), \
    StructField("block_hash", StringType(), True), \
    StructField("block_number", LongType(), True), \
    StructField("cumulative_gas_used", LongType(), True), \
    StructField("gas_used", LongType(), True), \
    StructField("contract_address", StringType(), True), \
    StructField("root", StringType(), True), \
    StructField("status", LongType(), True), \
    StructField("effective_gas_price", LongType(), True) \
  ])

In [7]:
tokens_schema = StructType([ \
    StructField("address", StringType(), True), \
    StructField("symbol", StringType(), True), \
    StructField("name", StringType(), True), \
    StructField("decimals", LongType(), True), \
    StructField("total_supply", LongType(), True), \
    StructField("block_number", LongType(), True), \
  ])

In [8]:
cmc_historical_schema = StructType([ \
    StructField("id", LongType(), True), \
    StructField("rank", LongType(), True), \
    StructField("name", StringType(), True), \
    StructField("symbol", StringType(), True), \
    StructField("open", DoubleType(), True), \
    StructField("high", DoubleType(), True), \
    StructField("low", DoubleType(), True), \
    StructField("close", DoubleType(), True), \
    StructField("volume", DoubleType(), True), \
    StructField("marketCap", DoubleType(), True), \
    StructField("timestamp", LongType(), True), \
    StructField("address", StringType(), True), \
  ])

In [9]:
cmc_address_schema = StructType([ \
    StructField("rank", LongType(), True), \
    StructField("bsc", StringType(), True), \
    StructField("eth", StringType(), True), \
    StructField("polygon", StringType(), True), \
  ])

In [9]:
base_path = "s3a:///bsc-spark-test"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
token_transfers_df = spark.read.format("csv") \
    .option("header", True) \
    .schema(token_transfers_schema) \
    .load(base_path + "/token_transfers/*/*/*.csv")

In [12]:
transactions_df = spark.read.format("csv") \
    .option("header", True) \
    .schema(transactions_schema) \
    .load(base_path + "/transactions/*/*/*.csv")

In [13]:
receipts_df = spark.read.format("csv") \
    .option("header", True) \
    .schema(receipts_schema) \
    .load(base_path + "/receipts/*/*/*.csv")

In [14]:
tokens_df = spark.read.format("csv") \
    .option("header", True) \
    .schema(tokens_schema) \
    .load(base_path + "/tokens/*.csv")

In [15]:
cmc_historicals_df = spark.read.format("csv") \
    .option("header", True) \
    .schema(cmc_historical_schema) \
    .load(base_path + "/cmc_historicals/*.csv")

In [16]:
cmc_addresses_df = spark.read.format("csv") \
    .option("header", True) \
    .schema(cmc_address_schema) \
    .load(base_path + "/cmc_addresses/*.csv")

In [17]:
from pyspark.sql.functions import col

In [18]:
transactions_df = transactions_df.drop(col("input"))

In [19]:
token_transfers_df.createOrReplaceTempView("token_transfers")
transactions_df.createOrReplaceTempView("transactions")
receipts_df.createOrReplaceTempView("receipts")
tokens_df.createOrReplaceTempView("tokens")
cmc_historicals_df.createOrReplaceTempView("cmc_historicals")
cmc_addresses_df.createOrReplaceTempView("cmc_addresses")

In [53]:
# change ETH, eth if other networks

import time

start_time = time.time()

result_df = spark.sql("""
SELECT 
    nt.block_number,
    nt.from_address,
    nt.to_address,
    nt.value,
    null as token_transfer,
    null as token_contract,
    ((nt.value / POWER(10, 18)) * cmc_h.open) AS volume,
    (r.gas_used * nt.gas_price) / POWER(10,18) as gas_spent,
    ((r.gas_used * nt.gas_price) / POWER(10,18)) * cmc_h.open as gas_spent_usd
FROM (
    SELECT tx.block_number,
          tx.from_address,
          tx.to_address,
          tx.gas,
          tx.gas_price,
          tx.value,
          tx.hash,
          tx.block_timestamp
    FROM transactions tx
    WHERE tx.value > 0
) nt
LEFT JOIN receipts r ON nt.hash = r.transaction_hash
CROSS JOIN cmc_addresses cmc_addr
LEFT JOIN cmc_historicals cmc_h ON cmc_addr.rank = cmc_h.rank
WHERE nt.block_timestamp < cmc_h.timestamp AND nt.block_timestamp >  cmc_h.timestamp - 86400 
    AND cmc_h.symbol = 'ETH'
UNION ALL
SELECT 
    tt.block_number,
    tt.from_address,
    tt.to_address,
    null as value,
    tt.value as token_transfer,
    tt.token_address,
    ((tt.value / POWER(10, t.decimals)) * cmc_h.open) AS volume,
    (r.gas_used * t.gas_price) / POWER(10,18) as gas_spent,
    ((r.gas_used * t.gas_price) / POWER(10,18)) * native_cmc_h.open as gas_spent_usd
FROM token_transfers tt
    LEFT JOIN transactions t ON tt.transaction_hash = t.hash
    LEFT JOIN receipts r ON tt.transaction_hash = r.transaction_hash
    LEFT JOIN tokens t ON LOWER(tt.token_address) = LOWER(t.address)
    LEFT JOIN (SELECT eth, rank FROM cmc_addresses GROUP BY eth, rank LIMIT 1) cmc_addr 
        ON tt.token_address = cmc_addr.eth
    LEFT JOIN cmc_historicals cmc_h 
        ON (
            cmc_addr.rank = cmc_h.rank AND 
            t.block_timestamp < cmc_h.timestamp AND 
            t.block_timestamp >  cmc_h.timestamp - 8640
        )
    LEFT JOIN (
        SELECT * FROM cmc_addresses
        LEFT JOIN cmc_historicals ON cmc_addresses.rank = cmc_historicals.rank
        ) native_cmc_h 
            ON native_cmc_h.symbol = 'ETH'
                AND t.block_timestamp < native_cmc_h.timestamp 
                AND t.block_timestamp >  native_cmc_h.timestamp - 86400
""").withColumn('volume', col('volume').cast(DecimalType(38, 10))) \
    .withColumn('gas_spent', col('gas_spent').cast(DecimalType(38, 10))) \
    .withColumn('gas_spent_usd', col('gas_spent_usd').cast(DecimalType(38, 10))) 


time.time() - start_time

0.06949162483215332

In [54]:
result_df.show(10, False)

+------------+------------------------------------------+------------------------------------------+-------------------+--------------+--------------+---------------+------------+-------------+
|block_number|from_address                              |to_address                                |value              |token_transfer|token_contract|volume         |gas_spent   |gas_spent_usd|
+------------+------------------------------------------+------------------------------------------+-------------------+--------------+--------------+---------------+------------+-------------+
|28600000    |0xdccf3b77da55107280bd850ea519df3705d1a75a|0x56b438417b7ee788ae65abd5e87245194531017d|109500000000000000 |null          |null          |200.5053026593 |0.0001260000|0.2307184305 |
|28600000    |0xa180fe01b906a1be37be6c534a3300785b20d947|0x96e7bc5f932bd407196e7a97f28e7f6a9fde772e|172370250000000000 |null          |null          |315.6269328375 |0.0001260000|0.2307184305 |
|28600000    |0xa95b83af96d0b8

In [55]:
result_df.count()

241877

In [22]:
import math

start_time = time.time()

result_df.repartition(1) \
    .write \
    .option("header",True) \
    .csv(base_path + "/pre_txns_" + str(math.floor(start_time)))

time.time() - start_time


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2691.6926729679108